# Overview

In [ ]:
library(reticulate)

In [ ]:
sklearn <- import("sklearn")
datasets <- import("sktime.datasets")

# Prepare data

### Load dataset

In [ ]:
arrow_head <- datasets$load_basic_motions(split="train", return_X_y=TRUE)

In [ ]:
x_train <- arrow_head[[1]]
y_train <- arrow_head[[2]]

### Truncate some of the time series in order to complicate the problem

- 33.(3)% of sequences left at 100% length
- 33.(3)% of sequences truncated to 85% length
- 33.(3)% of sequences truncated to 75% length

In [ ]:
time_series_count = length(x_train[[1]])            # number of time series in the dataset
time_series_length = length(x_train[1,][[1]][[1]])  # number of observations in each time serie
time_series_dims = length(x_train[1,])              # number of dimensions of each time serie

first_index = 1                                     # index of first element of time series with 100% length
second_index = ceiling(time_series_count / 3)       # index of first element of time series with 75% length
third_index = ceiling(2 * time_series_count / 3)    # index of first element of time series with 50% length

mask = 0 : (time_series_length - 1)
for (i in first_index : (second_index - 1))
    for (j in 1 : time_series_dims)
        x_train[i,][[j]][[1]] = x_train[i,][[j]][[1]][mask]

mask = sort(sample(0 : (time_series_length - 1), ceiling(85 * time_series_length / 100))) 
for (i in second_index : (third_index - 1))
    for (j in 1 : time_series_dims)
        x_train[i,][[j]][[1]] = x_train[i,][[j]][[1]][mask]

    
mask = sort(sample(0 : (time_series_length - 1), ceiling(75 * time_series_length / 100)))
for (i in third_index : time_series_count)
    for (j in 1 : time_series_dims) 
        x_train[i,][[j]][[1]] = x_train[i,][[j]][[1]][mask]

# ROCKET

### Standarize time series length by repeating observations

In [ ]:
fix_length <- function(x, expected_length) {
    current_length = length(x)
    
    if (current_length != expected_length) {
        x_args = strtoi(stringr::str_trim(names(x)), 10)
        y_args = as.vector(x)
        return(approx(x_args, y_args, 0:99)$y)
    }
    
    return(x)
}

In [ ]:
max_time_series_length = max(lengths(x_train[[1]]))
time_series_count = length(x_train[[1]])
time_series_dims = length(x_train[1,])

for (i in 1 : time_series_count)
    for (j in 1 : time_series_dims)
        x_train[i,][[j]][[1]] = fix_length(x_train[i,][[j]][[1]], max_time_series_length)





In [ ]:
lengths(x_train[[1]])

In [ ]:
utils <- import("sktime.utils.validation.panel")
a = utils$check_X(x_train, coerce_to_numpy=TRUE)


In [ ]:
##sample

set.seed(5)
num_kernels = 20000 # default value


timepoints = ncol(x_train)
columns = nrow(x_train)
